# [Part 2] Accessing Sentinel EO data

## Introduction

In this notebook we will demonstrate the use of GRASS GIS 8.2+ in combination with Python within a Jupyter Notebook environment. We will use some GRASS GIS modules and related Python libraries which facilitate scripting (`grass.script`) and which let us connect and interact with Jupyter Notebooks (`grass.jupyter`). 

In this notebook we'll process Sentinel-2 data: NDVI computation and some time series processing.

### Table of contents

0. Why Jupyter Notebooks and how to use them?
1. GRASS GIS & Python
2. Paths and variables, connecting to GRASS GIS backend
3. Data upload to notebook session
4. Initialization of GRASS GIS in the Jupyter notebook session
5. Creating an area of interest map
6. Importing geodata into GRASS GIS
7. Sentinel-2 processing overview
8. Computing NDVI
9. Time series data processing
10. Supervised Classification: RandomForest
11. What's next?

## 0. Why Jupyter Notebooks and how to use them?

Jupyter Notebooks are server-client applications that allow code written in a notebook document to be edited and executed through a web browser. They can be run on a local computer that does not require Internet access, as well as used to control computations on a remote server accessed via the Internet ([documentation](https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)).

Jupyter notebooks can be interactive and are run through a web browser. They provide the ability to combine live code, explanatory text, and computational results into a single document. Jupyter Notebooks can be easily shared as documents.

Jupyter Notebooks are 

* convenient for initial code development (prototyping)
* ideal for code segmentation with the ability to re-run segments
* able to store values of variables from already executed segments.

The notebook can be saved as an executable Python script in addition to the native `.ipynb` format, or exported to various documentation formats such as PDF or Sphinx RST with nice styling.

#### Editing and interactive use

Editing a Jupyter notebook is very easy: in the web browser, you can navigate between text or code blocks ("cells") using the mouse or keyboard shortcuts (see Menu > Help > Keyboard Shortcuts). You can execute small code segments cell by cell, save the notebook in its current state, or modify and recalculate cells or return them to their previous state. In addition to executable code cells, you can use Markdown in documentation cells to make them presentable to others.

And: there are now many Jupyter notebooks on the Internet for inspiration!


## 1. GRASS GIS & Python

<!-- This cell has been written by Veronica Andreo, https://veroandreo.gitlab.io/ -->

### Python library "grass.script"

The **grass.script** or GRASS GIS Python Scripting Library provides functions for calling GRASS modules within Python scripts as sub-processes. The most commonly used functions include:

- `run_command()`: used when the output of the modules is a raster or vector, no text type output is expected
- `read_command()`: used when the output of the modules is a text
- `parse_command()`: used with modules whose output can be converted to `key=value` pairs
- `write_command()`: used with modules that expect text input, either in the form of a file or from stdin

It also provides several wrapper functions for frequently used modules:

- To get info from a raster, script.raster.raster_info() is used: `gs.raster_info('dsm')`
- To get info of a vector, script.vector.vector_info() is used: `gs.vector_info('roads')`
- To list the raster in a location, script.core.list_grouped() is used: `gs.list_grouped(type=['raster'])`
- To obtain the computational region, script.core.region() is used: `gs.region()`

More info: https://grass.osgeo.org/grass-stable/manuals/libpython/script_intro.html

### Python library "grass.jupyter"

The **grass.jupyter** library improves the integration of GRASS and Jupyter, and provides different classes to visualize maps.

- `init()`: starts a GRASS session and sets up necessary environment variables
- `Map()`: 2D rendering
- `Map3D()`: 3D rendering
- `InteractiveMap()`: interactive visualization with folium
- `TimeSeriesMap()`: visualization for spatio-temporal data

More info: https://grass.osgeo.org/grass-stable/manuals/libpython/grass.jupyter.html

Once set, we will test if the installation works.

## 2. Paths and variables, connecting to GRASS GIS backend

For the ease of working in this notebook, we define some session variables.

In [ ]:
# Path variables

import os

# data directory
homedir=os.path.join(os.path.expanduser('~'), "odse_workshop2022")

# GRASS GIS related variables, adapt path to your's
grassdata=os.path.join(homedir, "grassdata")
location="odse_workshop2022"
mapset="PERMANENT"

# Sentinel-2 related directories, adapt path to your's
s2_data=os.path.join(homedir, "sentinel")
s2_timestamps=os.path.join(homedir, "sentinel-timestamps.txt")

# create directories if not already existing
!mkdir -p $grassdata
!mkdir -p $s2_data

# the variables are also accessible via Python
print(homedir)

# list content
!ls -la $homedir

#### Next we check the GRASS GIS installation on the updated system.

In [ ]:
# Next we test the installation: you will see version and copyright statements
!grass --version

### Create an empty GRASS GIS location and mapset

The GRASS GIS database comes with a structure of folders and subfolders. We create an initial empty folder structure, for now using the GRASS GIS command line (in future also possible with Python). To match the projection of the Sentinel-2 data of Greece used later on, we use [EPSG:32634](http://epsg.io/32634).

In [ ]:
# create the GRASS GIS location with CRS set to EPSG:32634, then exit
!grass -c epsg:32634 -e $homedir/grassdata/$location

## 3. Data upload to notebook session

Next we will get the following data sets: https://geo.fsv.cvut.cz/geoharmonizer/odse_workshop_2022/grass/geodata.zip

Here we transfer the ZIP file to our notebook instance with `wget`.

In [ ]:
# download workshop data into target directory homedir
!wget -c https://geo.fsv.cvut.cz/geoharmonizer/odse_workshop_2022/grass/geodata.zip -O $homedir/geodata.zip

# unpack workshop data into target directory homedir
!unzip -o -q -d $homedir $homedir/geodata.zip

# get extra OSM map
!wget -c https://data.neteler.org/tmp/osm_greece_landuse.gpkg -O $homedir/geodata/osm_greece_landuse.gpkg

print("List uploaded file(s) in target directory homedir:")
!ls $homedir

### The data structure is as follows:

- geodata/highways.gpkg - Greece
- geodata/odse_tiles.gpkg - Geo-harmonizer tiles, The Netherlands and Greece
- geodata/dtm_5606.tif - elevation model, Greece (30 km x 30 km)
- geodata/dtm_14580.tif - elevation model, The Netherlands (30 km x 30 km)
- geodata/osm_greece_landuse.gpkg - OpenStreetMap "landuse" extract, Greece
- geodata/t34sgh_20_60m/* - three Sentinel-2 scenes, Greece (reduced in size)

We will later on look at the data here in the Jupyter notebook session.

We are now settled and ready to start the GRASS GIS analysis part.

## 4. Initialization of GRASS GIS in the Jupyter notebook session

While we have an installation of GRASS GIS, we still need to connect to it in this session.

In [ ]:
import os
import subprocess
import sys

# We ask GRASS GIS where its Python packages are to be able to start them from the notebook
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import GRASS packages
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
session = gj.init(grassdata, location, mapset)

At this point we are in a running GRASS GIS session, let's verify.

In [ ]:
# show current GRASS GIS settings
print(gs.read_command("g.gisenv"))

## 5. Creating an area of interest map

Let's create a vector map defined by a 1km buffer around a tile. We first import the tiles vector map `odse_tiles.gpkg`. Note that the CRS of the tiles map (`"ETRS89 Lambert Azimutal Equal Area"`) differs from the current location's CRS (EPSG `32634`). It means that GRASS GIS will perform the reprojection of the data into the current location CRS.

In [ ]:
# import
gs.run_command("v.import", input=homedir+"/geodata/odse_tiles.gpkg", output="odse_tiles")

# extract Greece tile from vector map
gs.run_command("v.extract", input="odse_tiles", where="tile_id = 5606", output="odse_tile_greece")

# show attributes
gs.vector_db_select('odse_tile_greece')['values']

Next we create the 1km buffer around a selected tile, followed by a [Python API](https://grass.osgeo.org/grass-stable/manuals/libpython/grass.script.html?highlight=info%20vector#grass.script.vector.vector_info) metadata query.

In [ ]:
# buffer single tile
gs.run_command("v.buffer", input="odse_tile_greece", where="tile_id = 5606", distance="1000",
               output="tile_5606_1km")

# show metadata
gs.vector_info("tile_5606_1km")

## 6. Importing geodata into GRASS GIS

### Importing the highways vector map

Next we import the highways vector map `highways.gpkg`. Note that the CRS of highways (EPSG `4326`) differs from the current location's CRS (EPSG `32634`). It means that GRASS GIS will perform the reprojection of the data into the current location CRS.

In [ ]:
gs.run_command("v.import", input=homedir+"/geodata/highways.gpkg", output="highways")
gs.vector_info("highways")

### Importing the landuse vector map

Now we import the landuse vector map `osm_greece_landuse.gpkg` which is based on selected OpenStreetMap tags. Note that the CRS of highways (EPSG `4326`) differs from the current location's CRS (EPSG `32634`). It means that GRASS GIS will perform the reprojection of the data into the current location CRS.

In [ ]:
gs.run_command("v.import", input=homedir+"/geodata/osm_greece_landuse.gpkg", output="osm_greece_landuse")

# show topological metadata
gs.vector_info_topo("osm_greece_landuse")

### Display of imported maps

We make use of the `InteractiveMap` tool - interactive visualization with folium.

In [ ]:
greecemap = gj.InteractiveMap(width = 400, tiles="OpenStreetMap")
# greecemap.add_vector("odse_tile_greece")
# greecemap.add_vector("tile_5606_1km")
greecemap.add_vector("highways")
greecemap.add_vector("osm_greece_landuse")
greecemap.add_layer_control(position = "bottomright")
greecemap.show()

## 7. Sentinel-2 processing overview

There are plenty of libraries or tools which allow downloading
Sentinel products from [Copernicus Open Access Hub](https://scihub.copernicus.eu/).

For GRASS GIS there is the [i.sentinel](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.html) toolbox. It consists of six GRASS addon modules:

* [i.sentinel.download](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.download.html)
* [i.sentinel.import](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.import.html)
* [i.sentinel.preproc](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.preproc.html)
* [i.sentinel.mask](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.mask.html)
* [i.sentinel.coverage](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.coverage.html)
* [i.sentinel.parallel.download](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.parallel.download.html)

Addons modules can be easily installed via [g.extension](https://grass.osgeo.org/grass-stable/manuals/addons/g.extension.html).

In [ ]:
gs.run_command("g.extension", extension="i.sentinel")

Note that [i.sentinel.download](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.download.html) requires also the [sentinelsat library](https://pypi.python.org/pypi/sentinelsat) to be installed.

In [ ]:
!pip3 install sentinelsat

### Select and download of Sentinel-2 data

Pre-downloaded Sentinel-2 scenes are available in the [sample dataset](https://geo.fsv.cvut.cz/geoharmonizer/odse_workshop_2022/grass/geodata.zip). Workshop participants can skip the download and continue with section "Import Sentinel-2 data", see below.

#### Steps to select and download of S2 data (please use the pre-downloaded Sentinel-2 scenes instead!)

Let’s download suitable Sentinel products for our area of interest (AOI) and perform a NDVI calculation. AOI region is defined by `tile_5606_1km` created above.

[Sentinel-2 L2A products](https://www.sentinel-hub.com/blog/sentinel-2-l2a-products-available-sentinel-hub)
will be used to avoid computing atmospheric corrections. Let’s
search for the latest available product by means of [i.sentinel.download](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.download.html). Setting the `-l` flag, the result will only
be printed. The download procedure will be performed later. In order to
search and download Sentinel products from the Copernicus Open Access Hub,
you have to create an account first. See the manual page of [i.sentinel.download](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.download.html) module for details. Upload or create a new text
file in the data directory (homedir) named `esa_credentials.txt` containing two lines (username and password).

#### Note

To get username and password you need to register at the [Copernicus Open Access Hub](https://scihub.copernicus.eu/), see the [Register new account](https://scihub.copernicus.eu/dhus/#/self-registration) page for signing up.

In [ ]:
# list available Sentinel-2 L2A scenes for AOI
# note that we use parse_command() in order to intercept the output for display in the browser
gs.parse_command("i.sentinel.download", flags="l", map="tile_5606_1km", producttype="S2MSI2A",
               settings=homedir+"/esa_credentials.txt")

This should result into something like:

```bash
c48f04f2-81d1-4fcc-bb32-97405925d6d4 ... 2022-05-25T09:05:59Z  0% S2MSI2A 1.02 GB
4720c028-0163-4aba-92a9-3e53fc4047c5 ... 2022-05-15T09:05:59Z  1% S2MSI2A 985.18 MB
0eb10137-025e-402b-a050-7e2919602e61 ... 2022-05-15T09:05:59Z  1% S2MSI2A 1.01 GB
...
```

By default the module returns products for the last 60 days. Let’s change
the search period setting `start` and `end` options. We will also
limit products by `clouds` coverage percentage threshold and `sort` products
by ingestion date.


In [ ]:
gs.parse_command("i.sentinel.download", flags="l", map="tile_5606_1km",
               producttype="S2MSI2A", settings=homedir+"/esa_credentials.txt",
               start="2022-02-01", end="2022-05-31", clouds="5",
               sort="ingestiondate")

This should result in:

```bash
a119f7b0-0020-4229-b0fb-efd0f5bdfb37 ... 2022-02-19T09:10:21Z  0% S2MSI2A 1.02 GB
d6e4253d-80e3-49ff-899e-cee19a0ba935 ... 2022-02-19T09:10:21Z  0% S2MSI2A 998.70 MB
cdb37429-1c5d-4c67-8ceb-b6c68a33eef6 ... 2022-03-26T09:06:09Z  0% S2MSI2A 1.02 GB
...
```

#### Tip: If more products have been found, you can limit the amount with the `limit` option.


Let’s download the desired product(s). Just remove the `-l` flag and add the `output` option in order to define the path to the output directory where data should be saved.

In [ ]:
# note: s2_data directory and esa_credentials.txt have been defined above

## TO BE SKIPPED, we use pre-downloaded data below!
#print("Storing S2 data in <"+s2_data+"> - this takes a few minutes...")
#gs.run_command("i.sentinel.download", map="tile_5606_1km", producttype="S2MSI2A",
#               settings=homedir+"/esa_credentials.txt", start="2022-02-01", end="2022-05-31",
#               clouds="5", output=s2_data)

### Importing Sentinel-2 data

Before importing or linking Sentinel-2 data we print a list of
filtered raster files including projection match (second column, 1 for
match, otherwise 0). If the CRS of input data differs from the current location
consider reprojection (`-r`) or creating a new location for import.

*Important*: Data will be imported into the new location by means of the [i.sentinel.import](https://grass.osgeo.org/grass-stable/manuals/addons/i.sentinel.import.html) tool. The command will import **all** Sentinel bands from `input` directory
recursively. Before importing data let’s check content of the input directory by `-p` flag.

In [ ]:
# Check list of pre-downloaded Sentinel-2 scenes, with i.sentinel.import (-p: print)
gs.parse_command("i.sentinel.import", flags="p", input=homedir+"/geodata/t34sgh_20_60m")

This should result in (projection match in second column: 1 for match, otherwise 0):

```bash
.../R20m/T34SGH_20210828T090549_B03_20m.jp2 1 (EPSG: 32634)
.../R20m/T34SGH_20210828T090549_B04_20m.jp2 1 (EPSG: 32634)
.../R20m/T34SGH_20210828T090549_B8A_20m.jp2 1 (EPSG: 32634)
...
```

In the example below, we limit the S2 data import to the RGB and NIR bands (2, 3, 4, 8A) in 20m spatial resolution by the `pattern` option.

In [ ]:
gs.parse_command("i.sentinel.import", flags="p", input=homedir+"/geodata/t34sgh_20_60m", pattern="B(02|03|04|8A)_20m")

This should result in:

```bash
.../R20m/T34SGH_20210828T090549_B8A_20m.jp2 1 (EPSG: 32634)
.../R20m/T34SGH_20210828T090549_B04_20m.jp2 1 (EPSG: 32634)
.../R20m/T34SGH_20210624T090601_B04_20m.jp2 1 (EPSG: 32634)
.../R20m/T34SGH_20210624T090601_B8A_20m.jp2 1 (EPSG: 32634)
.../R20m/T34SGH_20210729T090559_B04_20m.jp2 1 (EPSG: 32634)
.../R20m/T34SGH_20210729T090559_B8A_20m.jp2 1 (EPSG: 32634)
```

By default, input data are imported into GRASS data format.
Alternatively, data can be linked if `-l` is given. It is also
useful to import cloud mask vector features by `-c` flag. We also use
`register_output` option to produce a timestamp plain text file
which will be used in [section 07](07.rst).

In [ ]:
# use 2GB of RAM for faster operations, s2_timestamps defined above
# this takes up to a few minutes...
gs.parse_command("i.sentinel.import", flags="c", input=homedir+"/geodata/t34sgh_20_60m",
               pattern="B(02|03|04|8A)_20m", memory=2000, register_output=s2_timestamps)

# print timestamp file for inspection
f = open(s2_timestamps, 'r')
content = f. read()
print(content)
f. close()

#### Semantic labels

A fairly new concept within GRASS GIS is semantic labels. These are especially relevant for satellite imagery as they allow us to identify to which sensor and band a given raster corresponds. These labels are particularly relevant when working with satellite image collections and also when classifying different scenes.

In [ ]:
# print semantic labels
for i in gs.list_grouped(type="raster")['PERMANENT']:
    label = gs.raster_info(map=i)['semantic_label']
    print('Map: {}, Semantic label: {}'.format(i,label))

## 8. Computing NDVI

To compute [NDVI - Normalized difference vegetation index](http://en.wikipedia.org/wiki/NDVI),
the red (VIS) and near-infrared (NIR) channels are required. In the case
of Sentinel-2, these are the 4th and 8th band, respectively.

<code>NDVI = (NIR - VIS) / (NIR  + VIS)</code>

Before computing, let’s set up the computational region based on the tile map.

In [ ]:
# note that we use parse_command() to immediately see the printed computational region
gs.parse_command("g.region", vector="tile_5606_1km", flags="p")
gs.run_command("v.in.region", output="greece_box")
gs.parse_command("g.region", vector="greece_box", align="T34SGH_20210624T090601_B04_20m", flags="p")

NDVI can be computed via the [i.vi](https://grass.osgeo.org/grass-stable/manuals/i.vi.html)
tool which allows computing various vegetation indices.

In [ ]:
gs.run_command("i.vi", red="T34SGH_20210624T090601_B04_20m", viname="ndvi",
               nir="T34SGH_20210624T090601_B8A_20m", output="ndvi")

Add semantic label "NDVI" to the resulting `ndvi` map:

In [ ]:
gs.run_command("r.support", map="ndvi", semantic_label="S2_NDVI")
gs.raster_info(map="ndvi")["semantic_label"]

### Displaying the NDVI map in Folium (Leaflet)

In [ ]:
# Display newly created NDVI map
ndvimap = gj.InteractiveMap(width = 400, tiles="OpenStreetMap")
ndvimap.add_raster("ndvi", opacity=0.8)
ndvimap.add_vector("tile_5606_1km")
# ndvimap.add_vector("greece_box")
ndvimap.add_layer_control(position = "bottomright")
ndvimap.show()

## 9. Time series data processing

### A few concepts of time series data processing in GRASS GIS

GRASS GIS offers specialized tools for spatio-temporal data
processing, see GRASS documentation [temporalintro](https://grass.osgeo.org/grass-stable/manuals/temporalintro.html) for details.

GRASS introduces three special data types that are designed to handle
time-series data:

* *Space-time raster datasets* (`strds`) for managing raster map
  time series.

* *Space-time 3D raster datasets* (`str3ds`) for managing 3D raster
  map time series.

* *Space-time vector datasets* (`stvds`) for managing vector map time
  series.

### Create space-time dataset

At this moment a new space-time dataset can be created by means of [t.create](https://grass.osgeo.org/grass-stable/manuals/t.create.html) and all imported Sentinel bands registered with [t.register](https://grass.osgeo.org/grass-stable/manuals/t.register.html).

In [ ]:
gs.run_command("t.create", output="s2_tile_5606", title="Sentinel L2A 2021", desc="Tile 5606")
gs.run_command("t.register", input="s2_tile_5606", file=s2_timestamps)

Let’s check basic metadata (see [t.info](https://grass.osgeo.org/grass-stable/manuals/t.info.html)) and list the registered maps ([t.rast.list](https://grass.osgeo.org/grass-stable/manuals/t.rast.list.html)).

In [ ]:
gs.parse_command("t.info", input="s2_tile_5606")

This should result in:
```bash
...
| ... ------ Space Time Raster Dataset ------- ...
...
| Id: ...... s2_tile_5606@PERMANENT
...
| ... ------ Absolute time ------- ...
...
| Start time:................. 2021-06-24 09:19:52.607078
| End time:................... 2021-08-28 09:19:49.080855
...
| ... ------ Metadata information ------- ...
...
| Number of registered maps:.. 12
```

### List registered bands in space-time cube

In [ ]:
gs.parse_command("t.rast.list", input="s2_tile_5606")

This should result in:

```bash
name|mapset|start_time|end_time
T34SGH_20210624T090601_B04_20m|PERMANENT|2021-06-24 09:19:52.607078|None
T34SGH_20210624T090601_B8A_20m|PERMANENT|2021-06-24 09:19:52.607078|None
...
T34SGH_20210729T090559_B04_20m|PERMANENT|2021-07-29 09:19:53.186492|None
T34SGH_20210729T090559_B8A_20m|PERMANENT|2021-07-29 09:19:53.186492|None
T34SGH_20210828T090549_B04_20m|PERMANENT|2021-08-28 09:19:49.080855|None
T34SGH_20210828T090549_B8A_20m|PERMANENT|2021-08-28 09:19:49.080855|None
```

### NDVI Space-Time computation

For NDVI computation 4th and 8th bands are required. Map algebra for spatio-temporal data is performed by [t.rast.algebra](https://grass.osgeo.org/grass-stable/manuals/t.rast.algebra.html) which requires bands separated into different spatio-temporal datasets. Such datasets can
be prepared by [t.rast.extract](https://grass.osgeo.org/grass-stable/manuals/t.rast.extract.html).

Note that data aren't replicated by this extraction as the space-time support in GRASS GIS is metadata based.

In [ ]:
gs.run_command("t.rast.extract", input="s2_tile_5606", where="name like '%B04%'", output="s2_b4")
gs.run_command("t.rast.extract", input="s2_tile_5606", where="name like '%B8A%'", output="s2_b8")

Let’s check content of the new datasets by [t.rast.list](https://grass.osgeo.org/grass-stable/manuals/t.rast.list.html).

In [ ]:
gs.parse_command("t.rast.list", input="s2_b4")

In [ ]:
gs.parse_command("t.rast.list", input="s2_b8")

Set the computational region by [g.region](https://grass.osgeo.org/grass-stable/manuals/g.region.html)
including a mask for the area of interest by [r.mask](https://grass.osgeo.org/grass-stable/manuals/r.mask.html).

In [ ]:
gs.parse_command("g.region", vector="tile_5606_1km", align="T34SGH_20210624T090601_B04_20m", flags="p")
gs.run_command("r.mask", vector="tile_5606_1km")

NOTE: NDVI (see above) computation on spatio-temporal datasets can be performed in parallel (`nproc` parameter).

In [ ]:
gs.run_command("t.rast.algebra", basename="ndvi", nproc="2",
               expression="ndvi = float(s2_b8 - s2_b4) / ( s2_b8 + s2_b4 )")

When computation is finished *ndvi* color table can be set with [t.rast.colors](https://grass.osgeo.org/grass-stable/manuals/t.rast.colors.html).

In [ ]:
gs.run_command("t.rast.colors", input="ndvi", color="ndvi")

#### Show metadata of NDVI time series

In [ ]:
gs.parse_command("t.info", input="ndvi")

### Query time series

In [ ]:
# query region center coordinates for query, in UTM34N
gs.parse_command("g.region", flags="c")

In [ ]:
# query map at center coordinates
gs.parse_command("t.rast.what", strds="ndvi", coordinates="748800,4226800", layout="col", flags="n")

### Display NDVI time series in Folium (Leaflet)


Note: [TimeSeriesMap()](https://grass.osgeo.org/grass-stable/manuals/libpython/grass.jupyter.html?highlight=timeseriesmap#module-grass.jupyter.timeseriesmap) of `grass.jupyter` is still experimental and under development.


In [ ]:
### TO BE SKIPPED - in GRASS GIS 8.2.0 it takes "forever", bugfix pending.


## reduce resolution for faster display of time series, save original first for later
#gs.parse_command("g.region", save="default_res")
#gs.parse_command("g.region", flags="pa", res=50)
# 
## Display newly created NDVI time series map
#ndviseries = gj.TimeSeriesMap(use_region=True)
#ndviseries.add_raster_series("ndvi", fill_gaps=False)
#ndviseries.d_legend(color="black", at=(10,40,2,6))
#ndviseries.d_barscale()
#ndviseries.show()  # Create TimeSlider

# optionally, write out to animated GIF
# ndviseries.save("image.gif")

## restore original region
#gs.parse_command("g.region", region="default_res")

## 10. Supervised Classification: RandomForest

We will now demonstrate the workflow to perform a supervised [RandomForest classification](https://en.wikipedia.org/wiki/Random_forest) from training polygons in GRASS GIS. First we inspect the available S2 input bands, just to remember their names.

In [ ]:
gs.parse_command("g.list", type="raster")

Show the R-G-B composite of our chosen Sentinel-2 scene.

In [ ]:
s2map = gj.Map(width=400, use_region=True)
# show as a static map
s2map.d_rgb(red="T34SGH_20210624T090601_B04_20m", 
              green="T34SGH_20210624T090601_B03_20m", 
              blue="T34SGH_20210624T090601_B02_20m")
s2map.show()

# ... this will expectedly look very dark. Color balancing next!

In [ ]:
# perform color auto-balancing for RGB bands (also other band combinations may be auto-balanced)
gs.run_command("i.colors.enhance", 
               red="T34SGH_20210624T090601_B04_20m",
               green="T34SGH_20210624T090601_B03_20m", 
               blue="T34SGH_20210624T090601_B02_20m",
               strength=90)

# display again
s2map = gj.Map(width=400, use_region=True)
s2map.d_rgb(red="T34SGH_20210624T090601_B04_20m", 
              green="T34SGH_20210624T090601_B03_20m", 
              blue="T34SGH_20210624T090601_B02_20m")
s2map.d_vect(map="osm_greece_landuse")
s2map.show()

#### Stack of maps = image group

When you work with a stack of raster maps (e.g., R-G-B channels or more) in GRASS GIS, you can best handle this stack by creating a group. It is just based on metadata, so it does not take up more disk space.

In [ ]:
# generate list of selected S2 maps
s2_maps = gs.list_grouped(type="raster", pattern="*20210624T090601*")['PERMANENT']
print(s2_maps)

# create group and subgroup with S2 bands
gs.run_command("i.group", group="s2", subgroup="s2", input=s2_maps)
gs.parse_command("i.group", group="s2", flags="l")

### Creation of a training map by sampling from existing data

We will perform stratified sampling on the `osm_greece_landuse` map which is an incomplete extract from [OpenStreetMap](https://www.openstreetmap.org/#map=11/38.2050/23.8266) by using selected tags (aiming at matching the CORINE legend).

As a first step we rasterize the map, matching the pixel geometry of the Sentinel-2 red band.

In [ ]:
# define pixel geometry for new raster map: extent from vector map,
# pixel position and size from Sentinel-2 20m band)
gs.parse_command("g.region", vector="osm_greece_landuse",
                 align="T34SGH_20210624T090601_B04_20m", flags="p")

In [ ]:
# list column names of vector map
gs.vector_columns("osm_greece_landuse")

In [ ]:
# now we convert the landuse map from vector to raster model to simplify the next steps
# (names may be identical since the data type is different)
gs.run_command("v.to.rast", input="osm_greece_landuse", output="osm_greece_landuse", 
               use="attr", attribute_column="CORINE_class", label_column="landuse")

#### Simplification of the training classes

Since we have (in this example) only a few Sentinel-2 bands available, we will simplify the legend prior to using the training data for the training of the model:

- 11 12 = 11 urban
- 13    = 13 agriculture
- 14 21 22 23 32 = 14 greenland
- 31    = 31 forest
- 51    = 51 water
- all other  = NULL

In [ ]:
# to simplify teaching life, we just download the class redefinition file
!wget -c https://data.neteler.org/tmp/lulc_rules.recl -O $homedir/lulc_rules.recl

In [ ]:
# Way 1: use Python
#rules = ["11 12 = 11 urban", "13 = 13 agriculture", "14 21 22 23 32 = 14 greenland", "31 = 31 forest", "* = NULL"]
#p = gs.feed_command("r.reclass", input="landuse_train", output="landuse_train_simplified2",
#                          rules="-")
#p.stdin.write("\n".join(rules).encode())
#p.stdin.close()

# Way 2: use the rules file which we downloaded
# raster map: apply class simplification
gs.run_command("r.reclass", input="osm_greece_landuse", output="osm_greece_landuse_simplified",
                 rules=homedir+"/lulc_rules.recl")

In [ ]:
# check raster categories of simplified training map
gs.parse_command("r.category", map="osm_greece_landuse_simplified", separator="comma")

In [ ]:
# set colors
colours = ["11 195:20:0", "13 255,255,168", "14 100:240:100", "31 77:105:0", "51 0:204:242"]
colourise = gs.feed_command("r.colors", map="osm_greece_landuse_simplified", rules="-", quiet=True)
colourise.stdin.write("\n".join(colours).encode())
colourise.stdin.close()

In [ ]:
# display newly created raster map
osmlulc = gj.InteractiveMap(width = 400, use_region=True, tiles="OpenStreetMap")
osmlulc.add_raster("osm_greece_landuse_simplified", opacity=0.8)
osmlulc.add_layer_control(position = "bottomright")
osmlulc.show()

In [ ]:
# show simple legend
legend = gj.Map(width=400, use_region=True)
# at=bottom,top,left,right, percentage of screen coordinates (0,0 is lower left)
legend.d_legend(raster="osm_greece_landuse_simplified", title="Classes",
                fontsize=20, at=(20, 80, 20, 80), flags="n")
legend.show()

### Random sampling from rasterized simplified landuse map

We now perform stratified sampling, i.e. we extract for each land use class `n` sampling points, using the GRASS GIS addon [r.sample.category](https://grass.osgeo.org/grass-stable/manuals/addons/r.sample.category.html).

First, we install this addon.

In [ ]:
gs.run_command("g.extension", extension="r.sample.category")

In [ ]:
# stratified random sampling, generated vector points
gs.run_command("r.sample.category", input="osm_greece_landuse_simplified", output="landuse_train", n="150")

In [ ]:
# display newly created vector points map
train = gj.InteractiveMap(width = 400, use_region=True)
train.add_raster("osm_greece_landuse_simplified", opacity=0.7)
train.add_vector("landuse_train")
train.add_layer_control(position = "bottomright")
train.show()

In [ ]:
# list column names of vector points map
gs.vector_columns("landuse_train", getDict=False)

In [ ]:
# show vector attribute table
gs.vector_db_select("landuse_train")

Since the machine learning classifier expects raster points, we convert the vector sampling points accordingly using  [v.to.rast](https://grass.osgeo.org/grass-stable/manuals/v.to.rast.html).

In [ ]:
# convert points from vector to raster model
# (names may remain identical since the data type is different)
gs.run_command("v.to.rast", input="landuse_train", output="landuse_train", 
               use="attr", attribute_column="osm_greece_landuse_simplified", label_column="label")

In [ ]:
# check raster categories
gs.parse_command("r.category", map="landuse_train", separator="comma")

In [ ]:
# Show univariate statistics
gs.parse_command("r.univar", map="landuse_train", flags="g")

In [ ]:
# display newly created raster map
# NOTE: zoom in to spot the raster sampling points ;-)
train = gj.InteractiveMap(width = 400, use_region=True)
train.add_raster("landuse_train", opacity=0.8)
train.add_layer_control(position = "bottomright")
train.show()

### Perform machine learning model training (RandomForest)

First we have to install the [r.learn.ml2](https://grass.osgeo.org/grass-stable/manuals/addons/r.learn.ml2.html) extention. It consists of two modules: `r.learn.train` and `r.learn.predict`.

In [ ]:
# install ML extension
gs.run_command("g.extension", extension="r.learn.ml2")

We now train the ML model using [r.learn.train](https://grass.osgeo.org/grass-stable/manuals/addons/r.learn.train.html), with model "RandomForestClassifier".

In [ ]:
# train a random forest classification model using r.learn.train
gs.run_command("r.learn.train", group="s2", training_map="landuse_train",
               model_name="RandomForestClassifier",
               n_estimators="500", save_model=homedir+"/rf_model.gz")

 The model has been stored in the file `rf_model.gz` for use in the prediction step of the supervised classification.

In [ ]:
!ls -la $homedir

### Perform ML supervised classification

The trained model will now be applied to the entire dataset.

In [ ]:
# perform prediction using r.learn.predict
gs.run_command("r.learn.predict", group="s2", load_model=homedir+"/rf_model.gz", output="sentinel_rf")

Add some styling: category labels and colors (colors inspired by CORINE land cover map legend)

class | R:G:B | label
- 11 195:20:0 urban
- 13 255,255,168 agriculture
- 14 100:240:100 greenland
- 31 77:105:0 forest
- 51 0:204:242 water

In [ ]:
# set colors
colours = ["11 195:20:0", "13 255,255,168", "14 100:240:100", "31 77:105:0", "51 0:204:242"]
colourise = gs.feed_command("r.colors", map="sentinel_rf", rules="-", quiet=True)
colourise.stdin.write("\n".join(colours).encode())
colourise.stdin.close()

With this the (oversimplified) supervised classification has been completed and we can display the result.

### Supervised classification: reporting and display

In [ ]:
# Display newly created sentinel_rf map
map = gj.InteractiveMap(width = 400, tiles="OpenStreetMap")
map.add_raster("sentinel_rf", opacity=0.8)
map.add_layer_control(position = "bottomright")
map.show()

In [ ]:
# show simple legend
legend = gj.Map(width=400, use_region=True)
# at=bottom,top,left,right, percentage of screen coordinates (0,0 is lower left)
legend.d_legend(raster="sentinel_rf", title="Classes",
                fontsize=20, at=(10, 90, 10, 90), flags="n")
legend.show()

In [ ]:
# Show class distribution in percent
gs.parse_command("r.report", map="sentinel_rf", units="p", flags="h")

In [ ]:
# export map to COG
gs.run_command("r.out.gdal", flags="fmt", input="sentinel_rf", output=homedir+"/greece_sentinel2_RF.tif",
               format="COG", overviews="4")

# see the output file
!ls -la $homedir

Keep in mind, this classification was just a simplified example to show how the procedure may work.

## 11. What's next?

You may enjoy more Jupyter notebooks at

https://github.com/OSGeo/grass/tree/main/doc/notebooks

### Talk to us

- Martin Landa, PhD, https://geo.fsv.cvut.cz/gwiki/Ing._Martin_Landa,_Ph.D.
- Markus Neteler, PhD, https://www.mundialis.de/en/neteler/